In [ ]:
%matplotlib inline
import numpy as np
import yt
yt.enable_parallelism()


In [ ]:
import sys
sys.path.append('/home/ychen/lib/util')
import util
import numpy as np
#import matplotlib
import logging
from matplotlib import pyplot as plt
import yt
yt.enable_parallelism()
import os
os.environ["OMP_NUM_THREADS"] = "8"

from tools import read_par, calcNozzleCoords

mu = 1.67E-24
k = 1.38E-16
T = 1.0E7
v = 0.1*3.0E10
mag = 5.0E-5

Extrema = { 'density': (1.0E-4*mu, 1.0E-2*mu), 'pressure':(1.0E-3*k*T, 1.0E1*k*T),'temperature': (1.0E1*T, 1.0E4*T),\
            'magnetic_field_x': (-mag, mag), 'magnetic_field_y': (-mag, mag), 'magnetic_field_z':(-mag,mag),\
            'velocity_x':(-v,v), 'velocity_y': (-v,v), 'velocity_z':(-v,v),\
            'ism ': (0.0, 1.0), 'jet ': (0.0, 1.0), 'magnetic_pressure': (1E-13, 1.0E-7),\
            'dens': (1.0E-5*mu, 1.0E1*mu), 'pres':(1.0E-5*k*T, 1.0E0*k*T),'temp': (10.0*T, 1.0E4*T),\
            'magx': (-mag, mag), 'magy': (-mag, mag), 'magz':(-mag,mag),\
            'velx': (-v,v), 'vely': (-v,v), 'velz':(-v,v),\
            'magp': (1E-20, 1.0E-7)\
}

logfield = { 'dens': True, 'pres': True, 'temp': True,\
             'velx': False, 'vely': False, 'velz': False,\
             'magx': False, 'magy': False, 'magz': False,\
             'ism ': False, 'jet ': False, 'magp': True,\
             'density': True, 'pressure': True, 'temperature': True,\
             'velocity_x': False, 'velocity_y': False, 'velocity_z': False,\
             'magnetic_field_x': False, 'magnetic_field_y': False, 'magnetic_field_z': False,\
             'magnetic_pressure': True}

# Scan for files

dir = '/home/ychen/d9/FLASH4/2015_production_runs/0529_L45_M10_b1_h1/'
pars = read_par(dir)

def rescan(printlist=False):
    files = util.scan_files(dir, '*hdf5_plt_cnt_[0-9][0-9][0-9][0-9]', printlist=printlist)
    return files
files = rescan(True)

In [ ]:
def render_field(ds, field, width=[1.0,1.0,1.0], L=None, l_max=None):
    # "cheat" and tell yt that your dataset is periodic
    ds.periodicity = (True, True, True)
    # Find the bounds in log space of for your field
    dd = ds.all_data()
    if field in Extrema:
        mi, ma = Extrema[field]
    else:
        print 'Finding Extrema'
        mi, ma = dd.quantities["Extrema"](field)[0]
    print "**** Rendering field:%15s: Min=%.3e Max=%.3e" % (field, mi, ma)
    if mi == ma: return
    
    log_scale = logfield[field]
    if log_scale:
        mi,ma = np.log10(mi), np.log10(ma)
    #mi = 0 if mi < 0 else mi

    # Instantiate the ColorTransferfunction.
    tf = yt.ColorTransferFunction((mi,ma))
    #tf.show()

    # Set up the camera parameters: center, looking direction, width, resolution
    #c = (pf.domain_right_edge + pf.domain_left_edge)/2.0 + np.array([-0.1/pf["unitary"], 0.0, 0.0])
    c = (ds.domain_right_edge + ds.domain_left_edge)/2.0
    # Line of Sight
    L = np.array([0.0, 0.0, 1.0]) if L is None else L
    print 'Line of Sight: ', L
    W = [width[0]*(pars['xmax']-pars['xmin']),\
         width[1]*(pars['ymax']-pars['ymin']),\
         width[2]*(pars['zmax']-pars['zmin'])]
    #W = ds.quan(0.2, 'unitary')
    
    N = 512

    # Create a camera object
    #print 'Creating a downsampling AMRKDTree l_max=%i' % l_max
    #kd = AMRKDTree(pf, l_max=l_max)
    print 'Creating a camera object, l_max=%i' % l_max
    cam = ds.camera(c, L, W, N, tf, fields = [field], log_fields = [log_scale], max_level=l_max, no_ghost=False)

    # Now let's add some isocontours, and take a snapshot, saving the image
    # to a file.
    tf.add_layers(40, 0.1)
    print 'Creating snapshot'
    #fname = '%s_L%.0f%.0f%.0f_rendering.png' % (field, L[0], L[1], L[2])
    im = cam.snapshot(num_threads=8)
    cam.show()
    return cam

#    for L in [np.array([0,0,1.]), np.array([1.,0,0]), np.array([0,1.,0]), np.array([1.,1.,1.,])]:
ds = yt.load(files[10].fullpath)
logging.getLogger().setLevel(logging.INFO)
L = np.array([0.,1.,0.2])
width = [0.5, 0.5, 0.5]
render = render_field(ds, 'density', width=width, L=L, l_max=4)